In [32]:
from ngsolve import *
from netgen.geom2d import SplineGeometry
from ngsolve.webgui import Draw

In [33]:
geo = SplineGeometry()
m = 0.6
geo.AddCircle(c=(0, 0), r=1, bc="cyl", maxh=m) # 0.04
mesh = Mesh(geo.GenerateMesh(maxh=m))
mesh2 = Mesh(geo.GenerateMesh(maxh=m))
mesh.Curve(3);
mesh2.Curve(3)
scene1 = Draw(mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': None…

In [34]:
# vector space for shape gradient
VEC = H1(mesh, order=1, dim=2)
# Test and trial functions
PHI, X = VEC.TnT()
gfX = GridFunction(VEC)
t = GridFunction(VEC)

In [35]:
surf_t = CoefficientFunction(1)
surf_0 = Integrate(surf_t,mesh)
# different try
#surf_0 = 4

dJOmega  = LinearForm(VEC)
vol = Parameter(1)
vol.Set(Integrate(surf_t,mesh))
alpha0 = 1
alpha = Parameter(alpha0)
dJOmega += ((alpha*((vol-surf_0)**2)*dx)).DiffShape(X) # (definedon="mesh_1")
#dJOmega += ((2*alpha*((vol-surf_0))*vol)*dx).DiffShape(X)
# vol-surf_0 = 1*dx() - zahl
# dJOmega += (surf_t*dx(definedon="mesh")-surf_t*dx(definedon="mesh2"))*((div(X))*dx)
# dJOmega += (surf_t*dx(definedon="mesh")-Parameter(surf_0))*((div(X))*dx)
#dJOmega += CoefficientFunction((x*y)).DiffShape(PSI)*dx

In [36]:
dJOmega.Assemble()
print(surf_0)
print(dJOmega.vec[0:5])

3.1417066211235207
       0
      0

       0
      0

       0
      0

       0
      0

       0
      0




In [37]:
mesh.UnsetDeformation()
test = CoefficientFunction((2*x,0.5*x))
t.Set(test)
mesh.SetDeformation(t)
scene = Draw(mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': None…

In [38]:
vol.Set(Integrate(surf_t,mesh))
dJOmega.Assemble()
print(vol.Get())
print(abs(surf_0-vol.Get()))
print(dJOmega.vec)

9.452133423394868
6.310426802271347
  10.1141
 -60.3608

  19.9045
 -3.46939e-15

  -10.1141
 60.3608

  -19.9045
 -5.45679e-15

  19.1681
 -53.5502

  23.0163
 -30.7793

  12.7525
 30.7793

  1.33523
 53.5502

  -19.1681
 53.5502

  -23.0163
 30.7793

  -12.7525
 -30.7793

  -1.33523
 -53.5502

  -5.32907e-15
 7.10543e-15

  -3.55271e-15
      0

       0
 -8.60423e-15

  4.66294e-15
 7.10543e-15

  3.55271e-15
 -5.60663e-15

  2.44249e-15
 7.10543e-15

  5.32907e-15
 -1.42109e-14




## try to do this as shape optimization

In [48]:
# don't use 2 deformations, just set surf_0 to something else, let's see if we get there:
mesh.UnsetDeformation()
surf_0 = 10

gfset = GridFunction(VEC)

dJOmega2 = LinearForm(VEC)
# this one doesn't work here
#dJOmega2 += ((alpha*((vol-surf_0)**2)*dx)).DiffShape(X) # (definedon="mesh_1")
# this one does work
#dJOmega2 += ((2*alpha*((vol-surf_0))*vol)*dx).DiffShape(X)
# this one works aswell:
dJOmega2 += ((2*alpha*((vol-surf_0))*(vol)*dx)).DiffShape(X)



b = BilinearForm(VEC)
b += (InnerProduct(grad(X),grad(PHI))).Compile() *dx + (InnerProduct(X,PHI)).Compile()*dx

def SolveDeformationEquation():
    rhs = gfX.vec.CreateVector()
    rhs.data = dJOmega2.vec - b.mat * gfX.vec
    update = gfX.vec.CreateVector()
    update.data = b.mat.Inverse(VEC.FreeDofs()) * rhs
    gfX.vec.data += update

In [49]:
# volume before
Integrate(surf_t,mesh)

3.1417066211235207

In [50]:
import time

gfset.Set((0,0))
mesh.SetDeformation(gfset)
scene.Redraw()
iter_max = 35

for i in range(0,iter_max):
    mesh.SetDeformation(gfset)
    scene.Redraw()
    
    b.Assemble()
    vol.Set(Integrate(surf_t,mesh)) # needed for multiple iterations (already done once before)
    dJOmega2.Assemble()
    SolveDeformationEquation()
    
    print(Integrate(surf_t,mesh))
    mesh.UnsetDeformation()
    
    gfxnorm = Norm(gfX.vec)
    scale = 0.1 / gfxnorm
    gfset.vec.data -= scale * gfX.vec
    time.sleep(0.3)

3.1417066211235207
3.3117426307357003
3.4863942872430136
3.6656740854858905
3.849594636215097
4.0381686511821675
4.231408928033909
4.429328335033572
4.631939795632392
4.839256272916322
5.051290753953879
5.268056234072002
5.489565701087749
5.715832119524542
5.946868414842416
6.182687457712414
6.423302048365796
6.668724901049226
6.918968628617404
7.174045727294831
7.4339685616384354
7.698749349732768
7.968400148649279
8.242932840200732
8.522359117021505
8.806690469003739
9.095938170118576
9.390113265650811
9.689226559874259
9.993288604193808
10.302309685779035
9.993030728456034
10.302043915104107
9.992772790806216
10.301778080339393


In [51]:
# volume after
mesh.SetDeformation(gfset)
Integrate(surf_t,mesh)

9.992514791248553